Imports

In [1]:
from google.colab import drive
import sys

drive.mount('/content/drive')
sys.path.append('/content/drive/Othercomputers/My Mac/src')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from torch import nn
from transformers import RobertaForSequenceClassification
import torch

from data_processing.persuasion_strategy_dataset import *
from data_processing.utils import *
from modeling.binary_classifier import BertClassifier
from modeling.train import train_model
from modeling.eval import eval_model
from modeling.stats import *
from modeling.test import test_model_text

# Gen Dataset

In [6]:
label = '5-AUTHORITY'
bin_df = pd.read_csv(f'/content/drive/Othercomputers/My Mac/binary_sentence_level/{label}_binary_sentence.csv')
tokenizer = 'roberta-base'
bin_df.head()

,text,label,binary_label,target_column
0,Hi.,8-NO-PERSUASION,0,5-AUTHORITY
1,not much.,8-NO-PERSUASION,0,5-AUTHORITY
2,do you donate to any charities,1-RAPPORT,0,5-AUTHORITY
3,Howmuch do you think you can donate,2-NEGOTIATE,0,5-AUTHORITY
4,That's great.,1-RAPPORT,0,5-AUTHORITY


In [4]:
train_dataset, test_dataset = gen_datasets(bin_df, label, tokenizer)
train_dataloader = gen_dataloaders(train_dataset, 32)
test_dataloader = gen_dataloaders(test_dataset, 1, test=True)

# Set Hyperparamters

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 10
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss for multilabel classification

model_name = 'roberta'
num_labels = 2

if model_name == 'roberta':
  model = RobertaForSequenceClassification.from_pretrained(tokenizer, num_labels=2)

else:
  model = BertClassifier('roberta', tokenizer, 2)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train

In [11]:
avg_loss, trained_model = train_model(model, train_dataloader, num_epochs,
  device, optimizer, criterion, model_name)

Average Epoch Loss:   0%|          | 0/10 [00:00<?, ?it/s]

Loss: 0:   0%|          | 0/217 [00:00<?, ?it/s]

KeyboardInterrupt: 

Evaluation

In [ ]:
true_labels, predicted_labels = eval_model(trained_model, test_dataloader, device)
accuracy, precision, recall, f1 = gen_stats(true_labels, predicted_labels, True)
report = gen_classification_report(true_labels, predicted_labels, True)

In [ ]:
test = 'you are making me feel really sad, I do not know why you would not do it'

pred = test_model_text(test, trained_model,
                    tokenizer, 'AUTHORITY', verbose=True)

'Not authority'

In [ ]:
torch.save(trained_model.state_dict(), 'authority_binary_model.pt')
